# Table of Contents
 - Introduction
 - Import Libraries
 - Importing and Data Preparation
 - Defining Process
 - Simulation Run (Part 1)
 - Defining Process
 - Optimisation (Part 2 - minimising travel time)

# Introduction
The purpose of this project is to simulate and optimise the High Speed 2 train running from London Euston to Birmingham Curzon Street.

The full line is divided by stations and signalling blocks. As per the provided details, there are in total 4 stations namely: 
+ London Euston
+ London Old Oak
+ Birmingham Interchange
+ Birmingham Curzon Street.

The simulation model of this train has been designed as per the give instructions, and details. The main part of this model is the creation of the dynamic dataset, which takes the details from a stationary dataset, and enumerates the stations and signalling blocks as different rows, instead of a single row. The second part is the use of Simpy, which makes each of the signalling block and station as a resource which the train has to request to enter, and if it is not free, then the train is asked to wait, till it becomes clear.
The optimisation has been done to find the minimum travel time required by the train, and the combination of number of trains and signalling blocks necessary to minimise the travel time.

# Import Libraries

In [1]:
import pandas as pd
import scipy.stats as stats
import math
import numpy as np
import itertools
import simpy
import csv
import simpy.events as evt
import random
import matplotlib.pyplot as plt
import statistics
from time import sleep

# Importing and Data Preparation

Following is the stationary dataset that has been created as a csv file and pulled into the notebook. Here, I have taken the From and To stations. The distance between the stations is assumed based on the real world distances. The travel time is set as per the given details in the question, i.e 5 mins between London Euston and London Old Oak, 31 mins between London Old Oak and Birmingham Interchange, and 9 mins between Birmingham Interchange and Birmingham Curzon Street. 

The delay times are basically the dwell times that the trains spend in each station. This data, as well, I have assumed to be 10, 40 and 15 seconds respectively. The signalling blocks as well are assumed mainly for the second and third stations. The range of assumption of the signalling block is 9-14. Here, it is set as 13.

In [2]:
#Stationary Dataset

data1=pd.read_csv('HS2.csv')
data1

,Line,From,To,Distance,Signalling Blocks,Travel Time,Delay
0,HS2,London Euston,London Old Oak,10000,1,300,10
1,HS2,London Old Oak,Birmingham Interchange,161000,13,1860,40
2,HS2,Birmingham Interchange,Birmingham Curzon Street,14000,2,540,15


### Creating a new dynamic dataframe
Here, I have created the Dynamic datafreame that I mentioned in the introduction part. First I have created an empty dataframe with just the required columns, and then I have populate it using the following code.
- This code takes the values of the signalling blocks, and splits the From and To stations to different signalling blocks.
- So, if there are 13 signalling blocks in between 2nd and 3rd stations, the dynamic dataset has 13 rows each having a signalling block in From and To columns.
- This however a crude method, eases up the whole process.

In [3]:
# Creating the empty dataframe which will be populated later

data2 = pd.DataFrame (columns = ['Line','From','To','Travel Time','Delay'])

In [4]:
data2

,Line,From,To,Travel Time,Delay


In [4]:
# Creating the Dynamic dataset
# The first if condition populates the rows between the first two stations
# The for loop under the second if loops through the number of signalling blocks between the 2nd and 3rd stations
# and creates the subsequent signalling block rows

for i in range(len(data1)):
    if data1["From"][i] == "London Euston":
        data2.at[i,'Line']="HS2"
        data2.at[i,'From']="London Euston"
        data2.at[i,'To']="SigBloc0"
        data2.at[i,'Travel Time']=0
        data2.at[i,'Delay']=10
        data2.at[i+1,'Line']="HS2"
        data2.at[i+1,'From']="SigBloc0"
        data2.at[i+1,'To']="London Old Oak"
        data2.at[i+1,'Travel Time']=300
        data2.at[i+1,'Delay']=0
    if data1["From"][i]=="London Old Oak":
        data2.at[i+1,'From']="London Old Oak"
        for n in range(data1["Signalling Blocks"][i]):
            data2.at[i+1+n,'To'] = f'SigBloc{1+n}'
            data2.at[i+1+n,'Travel Time']=1860//data1["Signalling Blocks"][i]
            data2.at[i+1+n,'Delay']=0
            data2.at[i+1+n,'Line']="HS2"
            data2.at[i+2+n,'From'] = f'SigBloc{1+n}'
            data2.at[i+2+n,'Travel Time']=1860//data1["Signalling Blocks"][i]
            data2.at[i+2+n,'Delay']=0
            data2.at[i+2+n,'Line']="HS2"
    if data1["From"][i]=="Birmingham Interchange":
        data2.at[n+2,'Line']="HS2"
        data2.at[n+2,'To']="Birmingham Interchange"
        data2.at[n+3,'From']="Birmingham Interchange"
        data2.at[n+3,'Travel Time']=540//2
        data2.at[n+3,'Delay']=0
        data2.at[n+3,'To']=f'SigBloc{data1["Signalling Blocks"][i]-1}'
        data2.at[n+4,'From']=f'SigBloc{data1["Signalling Blocks"][i]-1}'
        data2.at[n+4,'To']=f'SigBloc{data1["Signalling Blocks"][i]}'
        data2.at[n+4,'Travel Time']=540//2
        data2.at[n+4,'Delay']=0
        data2.at[n+4,'Line']="HS2"
        data2.at[n+5,'From']=f'SigBloc{data1["Signalling Blocks"][i]}'
        data2.at[n+5,'To']="Curzon Street"
        data2.at[n+5,'Travel Time']=540//2
        data2.at[n+5,'Delay']=0
        data2.at[n+5,'Line']="HS2"
        
# The following loop is to populate the delay or dwell times times for the stations London Old Oak and Birmingham Interchange

for i in range(len(data2)):
    if data2['From'][i]=="London Old Oak":
        data2.at[i,'Delay'] = 40
    if data2['From'][i]=="Birmingham Interchange":
        data2.at[i,'Delay'] = 15

##### Following is the dynamic dataframe crated

In [5]:
data2

,Line,From,To,Travel Time,Delay
0,HS2,London Euston,SigBloc0,0,10
1,HS2,SigBloc0,London Old Oak,300,0
2,HS2,London Old Oak,SigBloc1,143,40
3,HS2,SigBloc1,SigBloc2,143,0
4,HS2,SigBloc2,SigBloc3,143,0
5,HS2,SigBloc3,SigBloc4,143,0
6,HS2,SigBloc4,SigBloc5,143,0
7,HS2,SigBloc5,SigBloc6,143,0
8,HS2,SigBloc6,SigBloc7,143,0
9,HS2,SigBloc7,SigBloc8,143,0


#### Setting the time for the environment

In [6]:
def daytime(t):
    t=int(t)
    return f"{t//3600:02d}:{(t%3600)//60:02d}:{t%60:02d}"
def now():
    return daytime(env.now)

# Defining Process
This is the main process on which the simulation is based. The object Train is the base class, which is called by the Line function later on. This class consists of the constructor initializing all the variables. This makes sure that all the resources initialized in this class are available to all the trains that are running on the track.

The process function sets the running schedule of the train. I have used the simpy resource function to satisfy the 5 second rule of the signalling blocks. However, it also adds to the dwell time of the stations, which is acceptable, as the duration is less.

In [7]:
class Train(object):  
    def __init__(self, i,line, route):
        self.name = line+'-''[Train '+f"{i:3d}"+']'
        self.line=line
        self.route = route
        
# The direction has not been taken into account because the track is assumed to be one way at a given point in time.
   

    def process(self):
        here = self.route[0]   # starting location
        for dest in self.route[1:]:
            data=data2[data2['Line']==self.line][data2['From']==here]
            drivetime=data.iloc[0].at['Travel Time']
            dwelltime=data.iloc[0].at['Delay']
            yield env.timeout(dwelltime)
            print(f"{now():s} {self.name:s} dep {here:s} for {dest:s}")
            
# Using the simpy resource to generate the timeouts of each block
# This holds any train in the previous block if there is train in the current block
# And, takes a timeout of 5 seconds after the train leaves the current block, which simulates the lighting system
        
            sb = simpy.Resource(env, capacity = 1)         
            with sb.request() as req:
                yield req        
                yield env.timeout(5)
            here=dest
            yield env.timeout(drivetime)
            print(f"{now():s} {self.name:s} arr {here:s}")

#### The Line function
This is the main function that calls the train class methods, and sets the track for the train. Where the train class was the definition of the train, this function is the definition of the track.

The line here starts operating at 6 in the morning and one hour window has been taken for the sake of simulation. Each train starts from the first station at a time interval of 360 seconds on top of the passing time.

In [8]:
# The line function here hardcodes the start time as 6am, and stop time as 7 am. 
# This is for the 1 hour window that has been taken for the simulation. 
# Timing between start of two trains has been kept as 360, which means each train will start at an interval of 6 minutes, if the starting station is free.

def line(name='HS2', start=6*3600, stop=7*3600, timing=360):
    #sb = signallingBlocks()
    data=data2[data2['Line']==name]
    stations=data['From'].to_list()
    stations+=[data['To'].to_list()[-1]]  
    yield env.timeout(start-env.now) # the line starts operating at 6am
    for i in range(int((stop-start)/timing)):

# Train() class is called which integrates the trains to the tracks. Finally the line function is used to run the simulation.

        t=Train(i, name, stations)
        env.process(t.process())
        yield env.timeout(timing)

# Simulation Run
Following is the simulation that has been created. It shows the departure and arrival timings, and the simultaneous train numbers, which gives an idea of the movement of the trains on the track. Now, here I have assumed that the line is a single track, which means the trains will not be returning at the same time when it goes one way. This is why, the direction has not been considered in this simulation.

In [10]:
env = simpy.Environment()
env.process(line())
env.run()

06:00:10 HS2-[Train   0] dep London Euston for SigBloc0
06:00:15 HS2-[Train   0] arr SigBloc0
06:00:15 HS2-[Train   0] dep SigBloc0 for London Old Oak
06:05:20 HS2-[Train   0] arr London Old Oak
06:06:00 HS2-[Train   0] dep London Old Oak for SigBloc1
06:06:10 HS2-[Train   1] dep London Euston for SigBloc0
06:06:15 HS2-[Train   1] arr SigBloc0
06:06:15 HS2-[Train   1] dep SigBloc0 for London Old Oak
06:08:28 HS2-[Train   0] arr SigBloc1
06:08:28 HS2-[Train   0] dep SigBloc1 for SigBloc2
06:10:56 HS2-[Train   0] arr SigBloc2
06:10:56 HS2-[Train   0] dep SigBloc2 for SigBloc3
06:11:20 HS2-[Train   1] arr London Old Oak
06:12:00 HS2-[Train   1] dep London Old Oak for SigBloc1
06:12:10 HS2-[Train   2] dep London Euston for SigBloc0
06:12:15 HS2-[Train   2] arr SigBloc0
06:12:15 HS2-[Train   2] dep SigBloc0 for London Old Oak
06:13:24 HS2-[Train   0] arr SigBloc3
06:13:24 HS2-[Train   0] dep SigBloc3 for SigBloc4
06:14:28 HS2-[Train   1] arr SigBloc1
06:14:28 HS2-[Train   1] dep SigBloc1 fo

# Process Definition
The purpose of this part to optimise the total travel time based on the outputs of the simulation part. Here, I have assumed the range of signalling blocks from 1 to 14, and the number of trains from 1-15.

In [2]:
global BigPassengers
global SmallPassengers
BigPassengers=0
SmallPassengers=0
global Time
Time = 0 

class Train(object):
    def __init__(self,env,Ttrain_time=0,size=None,passengers=None):       
        self.env=env        
        self.Ttrain_time=Ttrain_time
        #----------------Train prop ----------------------        
        self.accel= 0.72                
        self.decel= 2.5       
        self.optimal_decel = 0.36        
        self.mxSpeed = 83.3        
        self.deceletrated_distance=0        
        self.number_of_blocks=0
        self.longest_line =145*1000# distance in meters
        self.remaining_distance=0        
        self.time_to_rest_block=0        
        self.randmaxSpeed=0        
        self.train_size=size               
        self.stations=None               
        self.passengers=passengers
        
    def time_taken_to_accelerate(self):
        #Accelaration starts once the decceleration and wait time is done
        self.randmaxSpeed=self.random_maximum_speed_of_choice()
        self.accelerated_distance=math.pow(self.randmaxSpeed,2)/(2*0.73)
        
        # time taken for acceleration
        self.accelerated_time = self.randmaxSpeed/self.accel
        # removing the distance covered during accelaration from the block
        self.remaining_distance-=self.accelerated_distance
        return self.accelerated_time                 
        
    def time_taken_to_decelerate(self,duration):
        # whether to apply emergency braking or not
        self.deceletrated_distance = math.pow(self.randmaxSpeed,2)/int(2*self.decel)
        
        #determining the signalling block length from the total number of blocks
        block_length=self.longest_line/self.number_of_blocks        
        self.remaining_distance = block_length - self.deceletrated_distance 
    
    def run_on_rail(self,blocks):        
        
        #time taken at the stations
        self.number_of_blocks= len(blocks) -3 # the three are the default one and last        
        self.arrive_time=self.env.now        
               
        if self.train_size!=None:
            self.stations=Stations(self.train_size)
                      
        for i,sigBlock in enumerate(blocks):
            
            with sigBlock.request() as request:
                before=self.env.now
                yield request #train requests for the first block
                wait_time=np.abs(self.env.now-before)
                print (f"Waiting time for free block :{np.round(wait_time)} seconds")
                
                #registering the wait time
                try:
                    self.time_taken_to_decelerate(wait_time)
                except:
                    pass
                
                yield self.env.timeout(np.abs(self.time_taken_to_accelerate()))                
                print (f"Accelaration time: {np.round(self.time_taken_to_accelerate())} seconds")                                                   
                yield self.env.timeout(np.abs(self.time_taken_to_pass_the_entire_block()))
                print (f"Waiting time for free block :{np.round(np.abs(self.time_taken_to_accelerate()))} seconds")                                  
               
        
        yield self.env.timeout(1)       
        if self.train_size!=None:
            yield self.env.timeout(self.stations.time_taken)
        
        #rout1.calculate_time()
        
        global Time        
        global BigPassengers
        global SmallPassengers
        
             
        Time+= np.abs(self.env.now-self.arrive_time)       
        if self.train_size!=None:
            if self.train_size=="Big":
                
                BigPassengers+= self.stations.stations_generate_time_and_people()[0]
               
                # Number of passangers taken by the train
                self.passengers["Big"]= BigPassengers
                print ("[BIG TRAIN] No. of passangers: " ,BigPassengers)
            if self.train_size=="Small":                                
                SmallPassengers+= self.stations.stations_generate_time_and_people()[0]                                
                self.passengers["Small"]=SmallPassengers                
                # Number of passangers taken by the train               
                print ("[SMALL TRAIN] No. of passangers: " ,SmallPassengers)                                           
        print( "Destination Reached")                
        
    def generate_stop_times(self):
        
        return  random.randint(1, 3)
    
    
    def time_taken_to_pass_the_entire_block(self):
        self.time_to_rest_block=self.remaining_distance/self.randmaxSpeed
        return self.time_to_rest_block
        
    def wind_and_storm_delay(self):        
         return  random.randint(0,1)
        
    def random_maximum_speed_of_choice(self):
        #choosing between max speeds 
        return random.randint(84,100)
    
class SignalBlock(simpy.resources.resource.Resource):
    def __init__(self,env,route):
        super().__init__(env)
        self.cross_time=route.cross_time
    def dummy(self):
        print("not present")
       
            
class Stations:
    def __init__(self,train_size):       
        self.time_taken = 0        
        self.train_size = train_size                        
            
    def generate_people_Birmingham_Curzon_Street(self):
        pass


    def generate_people_London_Euston(self):
        self.number_of_passangers+= random.randint(50,420) 
        if self.train_size == "Big":
            self.number_of_passangers+= random.randint(50,630)                     
            
           
        self.time_taken += 1/1.2 *self.number_of_passangers      
        return self.number_of_passangers

    def generate_people_London_Old_Oak_Common(self):

        # looping till the train reaches full capacity
        empty_seats= 420 - self.number_of_passangers 
        if self.train_size == "Big":
            empty_seats= 630 - self.number_of_passangers
        # Randomising the number of people who could have aligted
        empty_seats+= random.randint(0,420)
        if self.train_size == "Big":
            empty_seats+= random.randint(0,630)

        new_pass=random.randint(0,420)
        if self.train_size == "Big":
            new_pass=random.randint(0,630)

        while new_pass > empty_seats:

            new_pass= random.randint(0,420)
            if self.train_size == "Big":
                new_pass= random.randint(0,630)

        self.number_of_passanger= + new_pass
        self.time_taken += 1/1.2 *new_pass  
                  
        
class Route:
    def __init__(self,name):       
        self.cross_time=None                      
        if name is "LE -> LC":
            self.cross_time =  300 #5 mins x 60                        
        elif name is  "LC -> BI":
            self.cross_time =  1860 # 31 mins x 60        
        elif name is "BI -> BC":
            self.cross_time=  540 # 9 mins x 60            

class RailSystem(object):
    def __init__ (self,env, num_of_trains,blocks):        
        self.env=env
        self.num_of_trains = num_of_trains                
        self.signalBlocks = blocks       
      
        
    def run_this_train (self,train):      
        # waits for an empty signal block       
        # check if the first signal block if free so we put in a train              
        self.env.process(train.run_on_rail(self.signalBlocks))             
                  
        
def run_instance(env, num_of_trains=9,num_of_signaling_blocks=14):
            
    #  creating a list of signalling blocks
    blocks=[]
    num_of_signaling_blocks+=3
        
    for i in range(num_of_signaling_blocks):        
        signal_block = None       
        if i==0:
            route= Route("LE -> LC")
            signal_block=SignalBlock(env,route)
            
        elif i is (int)(num_of_signaling_blocks-2):
            route= Route("BI -> BC")
            signal_block=SignalBlock(env,route)
                         
                         
        elif i is (int)(num_of_signaling_blocks-1):
            route= Route("BI -> BC")
            signal_block=SignalBlock(env,route)
        else:
            route= Route("LC -> BI")
            signal_block=SignalBlock(env,route)           
                             
        blocks.append(signal_block)
               
    system = RailSystem(env, num_of_trains,blocks)
        
    # starting all the trains processes and checking if they could use the resources
    
    arrive_time = env.now
    Ttrain_time=0
    for i in range(num_of_trains):
        
        train = Train(env,Ttrain_time)
        system.run_this_train(train)
    end_time = env.now- arrive_time
    print(f"End {end_time}")
    
    # wait for all the processes to complete and 
    while True:
        yield env.timeout(0.20)
    



num_of_trains:str = 7
num_of_signaling_blocks :str = 14 # only consider the ones btween "BI -> BC" 
    
    

#### Dataset of different combinations of signalling blocks and train numbers

In [5]:
dt=pd.read_csv('Sig_blocks.csv')
dt.head(20)  #Total 150 combinations


,Signalling Blocks,Number of trains
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,8,1
7,10,1
8,13,1
9,14,1


# Optimisation (Part 2 - minimising travel time)

In [4]:
number_of_trains=[1, 2, 3, 4, 5, 6, 8, 10, 13, 14]
number_of_signal_block=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]



def optimal_nodes_and_trains(trains,blocks ) :
    global Time 
    Time=0
    pair = []  
    minimum_time=0    
    env = simpy.Environment()
       
    with open('Sig_blocks.csv', 'r') as file:
    
        csv_dict_reader = csv.DictReader(file)
        
        for i,row in enumerate(csv_dict_reader):    
            env.process(run_instance(env,int(row["Number of trains"]),int(row["Signalling Blocks"]))) 
            
            if len(pair)== 0 :
                pair.append(int(row["Signalling Blocks"]))
                pair.append(int(row["Number of trains"]))        
                minimum_time= (Time/int(row["Number of trains"])) + (0.5*60/int(row["Number of trains"]))
                        
            else:
                # check if the time is less than our minimum time
                if int(Time/int(row["Number of trains"]))< minimum_time:
                        pair.clear() 
                        pair.append(int(row["Signalling Blocks"]))
                        pair.append(int(row["Number of trains"]))                       
                        minimum_time = (Time/int(row["Number of trains"])) + (0.5*60/int(row["Number of trains"]))                               
        env.run(until=3600)
        return pair,minimum_time                                                                        

if __name__=="__main__":
                                                    
    pair,mmtime=optimal_nodes_and_trains(number_of_trains,number_of_signal_block)     
    print(f"Minimim time taken by: {pair} , {mmtime}")    
    print(f"Optimal number of signal blocks {pair[0]} with {pair[1]} trains within an hour")


End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
End 0
Waiting time for free block :0 seconds
Waiting time for free block :0 seconds
Waiting time for free 

# References
[1]M. Moarefdoost, "Optimization Modeling in Python: PuLp, Gurobi, and CPLEX", Medium, 2018. [Online]. Available: https://medium.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a. [Accessed: 05- May- 2020].

[2] R. Python, “SimPy: Simulating Real-World Processes With Python – Real Python.” https://realpython.com/simpy-simulating- with-python/ (Accessed: 03- May- 2020).

[3]W. Hart, "Python Optimization Modeling Objects (Pyomo)". [Online]. Available: https://www.researchgate.net/publication/226764811_Python_Optimization_Modeling_Objects_Pyomo. [Accessed: 01- May- 2020].

[4] “SimPy: Shared Resources – Stefan Scherfke.” https://stefan.sofa-rockers.org/2014/06/13/simpy-shared-resources/ (accessed 03- May- 2020).